In [2]:
import math
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from scipy.special import expit
from sklearn.model_selection import train_test_split

matplotlib.rcParams.update({'figure.figsize': (15, 9)})
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams.update({'axes.labelsize': 20})
matplotlib.rcParams.update({'xtick.labelsize': 12})
matplotlib.rcParams.update({'ytick.labelsize': 12})
matplotlib.rcParams.update({'font.family': 'Helvetica, Arial, sans-serif'})
%config InlineBackend.figure_format = 'retina'

sns.set_style("whitegrid")

# Machine Learning Lab 5: Evaluation and Multi-Layer Perceptron

#### Jake Rowland and Paul Herz
10/20/17

## 1. Introduction

### 1.1 Background

### 1.2 Business Case

### 1.3 Serviceability

## 2.The Datasheet: Preprocessing and Review

### 2.1 Datasheet Preparation

### 2.2 Datasheet Quality

### 2.3 Modifying the Datasheet for Classification Compatability

## 3. Comparing Perceptrons

### 3.1 Homespun Perceptrons

In [3]:
# Example adapted from https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch12/ch12.ipynb
# Original Author: Sebastian Raschka
# This is the optional book we use in the course, excellent intuitions and straightforward programming examples
# please note, however, that this code has been manipulated to reflect our assumptions and notation.
import numpy as np
from scipy.special import expit
import sys
import pandas as pd

# start with a simple base classifier, which can't be fit or predicted
# it only has internal classes to be used by classes that will subclass it
class TwoLayerPerceptronBase(object):
    def __init__(self, n_hidden=30,
                 C=0.0, epochs=500, eta=0.001, random_state=None):
        np.random.seed(random_state)
        self.n_hidden = n_hidden
        self.l2_C = C
        self.epochs = epochs
        self.eta = eta
        
    @staticmethod
    def _encode_labels(y):
        """Encode labels into one-hot representation"""
        onehot = pd.get_dummies(y).values.T
            
        return onehot

    def _initialize_weights(self):
        """Initialize weights with small random numbers."""
        W1_num_elems = (self.n_features_ + 1)*self.n_hidden
        W1 = np.random.uniform(-1.0, 1.0, size=W1_num_elems)
        W1 = W1.reshape(self.n_hidden, self.n_features_ + 1) # reshape to be W
        
        W2_num_elems = (self.n_hidden + 1)*self.n_output_
        W2 = np.random.uniform(-1.0, 1.0, size=W2_num_elems)
        W2 = W2.reshape(self.n_output_, self.n_hidden + 1)
        return W1, W2
    
    @staticmethod
    def _sigmoid(z):
        """Use scipy.special.expit to avoid overflow"""
        # 1.0 / (1.0 + np.exp(-z))
        return expit(z)
    
    @staticmethod
    def _add_bias_unit(X, how='column'):
        """Add bias unit (column or row of 1s) to array at index 0"""
        if how == 'column':
            ones = np.ones((X.shape[0], 1))
            X_new = np.hstack((ones, X))
        elif how == 'row':
            ones = np.ones((1, X.shape[1]))
            X_new = np.vstack((ones, X))
        return X_new
    
    @staticmethod
    def _L2_reg(lambda_, W1, W2):
        """Compute L2-regularization cost"""
        # only compute for non-bias terms
        return (lambda_/2.0) * np.sqrt(np.mean(W1[:, 1:] ** 2) + np.mean(W2[:, 1:] ** 2))
    
    def _cost(self,A3,Y_enc,W1,W2):
        '''Get the objective function value'''
        cost = np.mean((Y_enc-A3)**2)
        L2_term = self._L2_reg(self.l2_C, W1, W2)
        return cost + L2_term
    

In [6]:
class TwoLayerPerceptron(TwoLayerPerceptronBase):
    def _feedforward(self, X, W1, W2):
        """Compute feedforward step
        -----------
        X : Input layer with original features.
        W1: Weight matrix for input layer -> hidden layer.
        W2: Weight matrix for hidden layer -> output layer.
        ----------
        a1-a3 : activations into layer (or output layer)
        z1-z2 : layer inputs 

        """
        print('Shape of X:', X.shape)
        A1 = self._add_bias_unit(X, how='column')
        print('Shape of A1:', A1.shape)
        print('Shape of W1:', W1.shape)
        Z1 = W1 @ A1.T
        print('Shape of Z1:', Z1.shape)
        A2 = self._sigmoid(Z1)
        A2 = self._add_bias_unit(A2, how='row')
        print('Shape of A2:', A2.shape)
        print('Shape of W2:', W2.shape)
        Z2 = W2 @ A2
        print('Shape of Z2:', Z2.shape)
        A3 = self._sigmoid(Z2)
        print('Shape of A3:', A3.shape)
        return A1, Z1, A2, Z2, A3
    
    def _get_gradient(self, A1, A2, A3, Z1, Z2, Y_enc, W1, W2):
        """ Compute gradient step using backpropagation.
        """
        # backpropagation
        grad1 = np.zeros(W1.shape)
        grad2 = np.zeros(W2.shape)
        
        # for each instance's activations 
        for (a1,a2,a3,y) in zip(A1,A2.T,A3.T,Y_enc.T):
            dJ_dz2 = -2*(y - a3)*a3*(1-a3)
            dJ_dz1 = dJ_dz2 @ W2 @ np.diag(a2*(1-a2))
                         
            grad2 += dJ_dz2[:,np.newaxis]  @ a2[np.newaxis,:]
            grad1 += dJ_dz1[1:,np.newaxis] @ a1[np.newaxis,:] 
            # don't incorporate bias term in the z1 gradient 
            # (its added in a2 from another layer)
            

        # regularize weights that are not bias terms
        grad1[:, 1:] += (W1[:, 1:] * self.l2_C)
        grad2[:, 1:] += (W2[:, 1:] * self.l2_C)

        return grad1, grad2
    
    def predict(self, X):
        """Predict class labels"""
        _, _, _, _, A3 = self._feedforward(X, self.W1, self.W2)
        y_pred = np.argmax(A3, axis=0)
        return y_pred
    
    
    def fit(self, X, y, print_progress=False):
        """ Learn weights from training data."""
        
        X_data, y_data = X.copy(), y.copy()
        Y_enc = self._encode_labels(y)
        
        # init weights and setup matrices
        self.n_features_ = X_data.shape[1]
        self.n_output_ = Y_enc.shape[0]
        self.W1, self.W2 = self._initialize_weights()

        self.cost_ = []
        for i in range(self.epochs):

            if print_progress>0 and (i+1)%print_progress==0:
                sys.stderr.write('\rEpoch: %d/%d' % (i+1, self.epochs))
                sys.stderr.flush()

            # feedforward all instances
            A1, Z1, A2, Z2, A3 = self._feedforward(X_data,self.W1,self.W2)
            
            cost = self._cost(A3,Y_enc,self.W1,self.W2)
            self.cost_.append(cost)

            # compute gradient via backpropagation
            grad1, grad2 = self._get_gradient(A1=A1, A2=A2, A3=A3, Z1=Z1, Z2=Z2, Y_enc=Y_enc,
                                              W1=self.W1, W2=self.W2)

            self.W1 -= self.eta * grad1
            self.W2 -= self.eta * grad2
            

        return self

In [7]:
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

ds = load_digits()
X = ds.data/16.0
y = ds.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)


params = dict(n_hidden=50, 
              C=0.1, # tradeoff L2 regularizer
              epochs=200, # iterations
              eta=0.001,  # learning rate
              random_state=1)


nn = TwoLayerPerceptron(**params)
nn.fit(X_train, y_train, print_progress=1)
yhat = nn.predict(X_test)

Epoch: 4/200

Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)


Epoch: 8/200

Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)


Epoch: 12/200

...

Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)


Epoch: 196/200

Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)


Epoch: 200/200

Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (1437, 64)
Shape of A1: (1437, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 1437)
Shape of A2: (51, 1437)
Shape of W2: (10, 51)
Shape of Z2: (10, 1437)
Shape of A3: (10, 1437)
Shape of X: (360, 64)
Shape of A1: (360, 65)
Shape of W1: (50, 65)
Shape of Z1: (50, 360)
Shape of A2: (51, 360)
Shape of W2: (10, 51)
Shape of Z2: (10, 360)
Shape of A3: (10, 360)


### 3.2 Sklearn's Perecptrons

### 3.3 Observations